In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import codecs, json
from tqdm import tqdm
import numpy as np
from datetime import datetime
import time
import math
!pip install mne
from mne.decoding import CSP
import mne

import sys
sys.path.append('drive/MyDrive/BmiResearch')
from constants import constants

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 10.4 MB/s eta 0:00:00


In [ ]:
constants.SIGNAL_PROCESSING

['SLF', 'ASR', 'NOSP']

In [ ]:
constants.BASE_DATASET_PATH

'drive/MyDrive/BmiResearch/data/datasets/preprocessed/a_walk_in_the_park/baseline_chunked_data'

In [ ]:
def reconstruct_timeseries(chunks, size, overlap):
  stride = size - overlap
  final_series_list = []
  for chunk in chunks:
    final_series_list.append(chunk.T[0:stride])
  result = np.concatenate(final_series_list, axis = 0)
  return result


output_path:  drive/MyDrive/BmiResearch/data/datasets/preprocessed/a_walk_in_the_park/SLF_CSP

actual_base_dataset_path =  drive/MyDrive/BmiResearch/data/datasets/preprocessed/a_walk_in_the_park/SLF

sub-006, sub-009, sub-011, sub-013, 'sub-016', 'sub-017'

at fit: ValueError: array must not contain infs or NaNs



In [ ]:
for sp in ['SLF', 'ASR', 'NOSP']:
  output_path = f'{constants.PREPROCESSED_DATASET_PATH}/{sp}_CSP'
  print('output_path: ', output_path)

  if sp == 'NOSP':
    actual_base_dataset_path = constants.BASE_DATASET_PATH
  else:
    actual_base_dataset_path = f'{constants.PREPROCESSED_DATASET_PATH}/{sp}'
  print("actual_base_dataset_path = ", actual_base_dataset_path)

  for subject in sorted(os.listdir(actual_base_dataset_path)):
    print(subject)
    experiment_settings = dict()
    experiment_settings['general_params'] = {'low_filter':constants.low_filter,
                                            'high_filter':constants.high_filter,
                                            'frequency':constants.freq,
                                            'minutes_for_test':constants.minutes_for_test,
                                            'window_size':constants.window_size,
                                            'overlap':constants.overlap,
                                            'EEG_CHANNELS':constants.EEG_CHANNELS}
    experiment_settings['feature_extraction'] = 'CSP'
    experiment_settings['DateTime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    experiment_settings['BASE_DATASET_PATH'] = constants.BASE_DATASET_PATH
    experiment_settings['OUTPUT_PATH'] = output_path

    output_path_subject = (f'{output_path}/{subject}')
    os.makedirs(output_path_subject)

    experiment_settings[subject] = dict()

    csp = CSP(n_components=25, reg=None, log=True, norm_trace=False)

    # process train
    chanks_train = codecs.open(f'{actual_base_dataset_path}/{subject}/X_train_chunks.json', 'r', encoding='utf-8').read()
    chanks_train = json.loads(chanks_train)
    chanks_train = np.array(chanks_train)
    print("chanks_train = ", chanks_train.shape)

    final_y_train_list = codecs.open(f'{constants.BASE_DATASET_PATH}/{subject}/y_train_chunks.json', 'r', encoding='utf-8').read()
    final_y_train_list = json.loads(final_y_train_list)
    final_y_train_list = np.array(final_y_train_list)


    csp.fit(chanks_train, final_y_train_list)
    csp_filters = csp.filters_

    chanks_train_csp = np.asarray([np.dot(csp_filters, epoch) for epoch in chanks_train])

    train_reconstracted = reconstruct_timeseries(chanks_train_csp, 100, 80)
    train_reconstracted = train_reconstracted.tolist()
    json.dump(train_reconstracted, codecs.open(f'{output_path_subject}/X_fit.json', 'w', encoding='utf-8'),
    separators=(',', ':'),
    sort_keys=True,
    indent=4)

    chanks_train_csp = chanks_train_csp.tolist()
    json.dump(chanks_train_csp, codecs.open(f'{output_path_subject}/X_train_chunks.json', 'w', encoding='utf-8'),
      separators=(',', ':'),
      sort_keys=True,
      indent=4)

    # process test
    chanks_test = codecs.open(f'{actual_base_dataset_path}/{subject}/X_test_chunks.json', 'r', encoding='utf-8').read()
    chanks_test = json.loads(chanks_test)
    chanks_test = np.array(chanks_test)

    starttime = time.perf_counter()
    chanks_test_csp = np.asarray([np.dot(csp_filters, epoch) for epoch in chanks_test])
    proc_end = time.perf_counter() - starttime
    proc_1ch_s = round(proc_end / chanks_test_csp.shape[0], 5)
    experiment_settings[subject]['proc_1_test_ch_s'] = proc_1ch_s
    experiment_settings[subject]['proc_test_s'] = round(proc_end, 5)
    experiment_settings[subject]['len_test'] = chanks_test.shape[0]
    experiment_settings[subject]['output_path_subject'] = output_path_subject

    chanks_test_csp = chanks_test_csp.tolist()
    json.dump(chanks_test_csp, codecs.open(f'{output_path_subject}/X_test_chunks.json', 'w', encoding='utf-8'),
      separators=(',', ':'),
      sort_keys=True,
      indent=4)

    json.dump(experiment_settings, codecs.open(f'{output_path_subject}/experiment_settings.json', 'w', encoding='utf-8'),
      separators=(',', ':'),
      sort_keys=True,
      indent=4)
